In [2]:
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
with h5py.File("/home/ahojel/visual_prompting_vid/output_dir/isolated_latents_new/dataset.hdf5", 'r') as h5file: 
    encoder_metric_holder = []
    first_element = next(iter(h5file.values()))
    labels = list(first_element.keys())
    

In [4]:
labels

['Baseline', 'left_half', 'one', 'three', 'top_half', 'two']

In [ ]:
labels = ['left_half', 'one', 'three', 'top_half', 'two']

In [ ]:
labels

In [17]:
with h5py.File("/home/ahojel/visual_prompting_vid/output_dir/isolated_latents_new/dataset.hdf5", 'r') as h5file: 
    delta_latent_holder = []
    label_holder = []

    for i in tqdm(range(len(labels))):
        for pass_name, pass_group in h5file.items():
            current_latent = pass_group[labels[i]]['latent'][1]
            if labels[i] == "top_half":
                delta_latent_holder.append(current_latent)
                label_holder.append("avg"+labels[i]+"_unregularized")
                delta_latent = -current_latent + pass_group["Baseline"]['latent'][1]
            elif labels[i] != "Baseline":
                delta_latent = -current_latent + pass_group["Baseline"]['latent'][1]
            else:
                delta_latent = current_latent
            delta_latent_holder.append(delta_latent)
            label_holder.append(labels[i])

    

100%|██████████| 6/6 [00:03<00:00,  1.68it/s]


In [18]:
delta_latent_holder[0].shape

(1, 197, 512)

In [19]:
delta_latent_holder = np.array(delta_latent_holder)
delta_latent_holder.shape

(686, 1, 197, 512)

In [20]:
avg_tensors = {}
unique_labels = np.unique(label_holder)
for label in unique_labels:
    indices = [i for i, x in enumerate(label_holder) if x == label]
    avg_tensor = np.mean([delta_latent_holder[i] for i in indices], axis=0)
    avg_tensors[label] = avg_tensor


In [21]:
avg_tensors.keys()

dict_keys(['Baseline', 'avgtop_half_unregularized', 'left_half', 'one', 'three', 'top_half', 'two'])

In [22]:
def calculate_quadrant_indices(rows, cols, quadrant):
    """
    Calculate the start and end indices for each quadrant in the flattened tensor.
    """
    row_start, row_end = 0, 7
    col_start, col_end = 0, 7
    
    if quadrant == 2:  # Top Right
        col_start, col_end = 7, 14
    elif quadrant == 3:  # Bottom Left
        row_start, row_end = 7, 14
    elif quadrant == 4:  # Bottom Right
        row_start, row_end = 7, 14
        col_start, col_end = 7, 14

    indices = []
    for row in range(row_start, row_end):
        for col in range(col_start, col_end):
            index = row * rows + col
            indices.append(index)
    
    return indices


In [23]:
avg_tensors['Baseline'].shape

(1, 197, 512)

In [24]:
indices = [i for i, x in enumerate(label_holder) if x == "Baseline"]
latent_subset = np.array([delta_latent_holder[i] for i in indices])

# Find the element in latent subset which is closest to avg_tensors['label_Baseline']
distances = np.linalg.norm(latent_subset - avg_tensors['Baseline'], axis=(2,3))
closest_index = np.argmin(distances)
closest_element = latent_subset[closest_index]
avg_tensors["closest_baseline"] = closest_element

In [25]:
indices = [i for i, x in enumerate(label_holder) if x == "avgtop_half_unregularized"]
latent_subset = np.array([delta_latent_holder[i] for i in indices])

# Find the element in latent subset which is closest to avg_tensors['avgtop_half_unregularized']
# Find the element in latent subset which is closest to avg_tensors['label_Baseline']
distances = np.linalg.norm(latent_subset - avg_tensors['avgtop_half_unregularized'], axis=(2,3))
closest_index = np.argmin(distances)
closest_element = latent_subset[closest_index]
avg_tensors["closest_top_half"] = closest_element

In [26]:
avg_tensors["avg_baseline_top_half"] = avg_tensors["Baseline"]
avg_tensors["closest_baseline_top_half"] = avg_tensors["closest_baseline"]

In [27]:
unique_labels = np.unique(label_holder+["closest_top_half", "avg_baseline_top_half", "closest_baseline_top_half"])
for label in unique_labels:
    if label == "Baseline":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 2) + calculate_quadrant_indices(14, 14, 3) + calculate_quadrant_indices(14, 14, 4)
    elif label == "left_half":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 3)
    elif label == "top_half":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 2)
    elif label == "avg_baseline_top_half":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 2)
    elif label == "closest_baseline_top_half":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 2)
    elif label == "closest_top_half":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1) + calculate_quadrant_indices(14, 14, 2)
    elif label == "one":
        indices_to_keep = calculate_quadrant_indices(14, 14, 1)
    elif label == "two":
        indices_to_keep = calculate_quadrant_indices(14, 14, 2)
    elif label == "three":
        indices_to_keep = calculate_quadrant_indices(14, 14, 3)
        
    # Create a zero tensor
    zero_tensor = np.zeros_like(avg_tensors[label])
    
    # Replace the values at the indices_to_keep with the avg_tensor values
    zero_tensor[:, indices_to_keep] = avg_tensors[label][:, indices_to_keep]
    
    # Assign the modified tensor back to avg_tensors
    avg_tensors[label] = zero_tensor



In [28]:
decoder_holder = avg_tensors

In [16]:
neck_holder = avg_tensors

In [29]:
holder = {"neck":neck_holder, "decoder":decoder_holder}


In [30]:
holder.keys()

dict_keys(['neck', 'decoder'])

In [31]:
import pickle

# Save avg_tensors dictionary as a pickle file
with open('vectors/isolated_tokens_new.pkl', 'wb') as file:
    pickle.dump(holder, file)
